In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 845.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


### GPT 입력값 만들기

GPT 입력값을 만들려면 토크나이저부터 준비해야 하며 아래 코드를 수행하면 GPT 모델이 사용하는 토크나이저를 초기화할 수 있다. 먼저 자신의 구글 드라이브 경로(/gdrive/My Drive/nlpbook/bbpe)에는 이전 실습에서 만든 바이트 기준 BPE 어휘 집합(vocab.json)과 바이그램 쌍의 병합 우선순위(merge.txt)가 있어야 합니다.

In [ ]:
# GPT 토크나이저 선언: GPT2Tokenizer'는 텍스트를 GPT-2 모델이 이해할 수 있는 형식으로 변환
# 여기에는 텍스트를 토큰(단어 또는 하위 단어)으로 분할하고 이러한 토큰을 모델이 처리에 사용하는 숫자 ID로 변환하는 작업이 포함
from transformers import GPT2Tokenizer # transformers 라이브러리에서 GPT2Tokenizer 클래스를 가져온다.
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe") # 사전 훈련된 토크나이저 로드
tokenizer_gpt.pad_token = "[PAD]" # 토크나이저는 패딩 목적으로 사용할 토큰을 알아야 하기 때문에 'pad_token'을 명시적으로 설정하는 것이 필요

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
# 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
# GPT 모델은 바이트 기준 BPE를 적용
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

GPT 모델 입력 만들기

In [ ]:
#  GPT 모델 입력
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length", # 문자의 최대 길이에 맞춰 패딩. max_length 매개변수로 지정된 최대 길이로 균일하게 시퀀스를 채우도록 지시
    max_length=12, # 문장의 토큰 기준 최대 길이. 문장이 12개 이상의 토큰으로 변환되면 잘림. 더 적은 수로 변환되면 패딩 토큰으로 채워진다.
    truncation=True, # 문장 잘림 허용 옵션
)

`batch_inputs`의 내용을 확인
- input_ids: 문장의 토큰화된 표현을 포함하며, 각 토큰은 토크나이저 어휘에서 해당하는 ID로 대체.
- attention_mask: 각 위치에 토큰이 존재하는지를 나타내는 이진 마스크입니다. 토큰이 존재하면 1, 토큰이 존재하지 않으면(즉, 패딩인 경우) 0입니다. 이는 모델이 실제 데이터와 패딩을 구별할 수 있도록 도와준다.

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
# 'input_ids'는 토큰화 결과를 가지고 각 토큰들을 인덱스(index)로 바꾼 것
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# attention_mask는 일반 토큰이 자리한 곳(1)과 패딩 토큰이 자리한 곳(0)을 구분해 알려주는 장치
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
# text = '근육이 커지기 위해서는'
text = '이 영화는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

이 영화는 '내 이름은 김삼순'이었다.
김영화는 "그동안 내가 살아온 삶의 궤적을 돌아보면서 그 속에서 내 삶을 되돌아보는 시간을 가져보고 싶었다"고 말했다.
그는 이어 "이번 영화제에서는 관객들이 직접 영화를 보고 느낄 수 있는 기회를 마련했다"며 기대감을 나타냈다.
이날 상영된 작품은 총 4편.</d> 지난해 12월 31일부터 올 1월 1일까지 진행된 이번 공모전은 ‘2018 대한민국 디자인대상’ 수상작과 우수작을 선정하는 방식으로 진행됐다.
공모전 주제는 △디자인에 대한 관심과 열정, 창의성, 혁신성 등 3개 분야다.
수상작은 ▲‘디자인의 미래, 우리


KoGPT2 (한국어 GPT-2) Ver 2.0

In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2",bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer(
    sentences,
    padding="max_length", # 문자의 최대 길이에 맞춰 패딩. max_length 매개변수로 지정된 최대 길이로 균일하게 시퀀스를 채우도록 지시
    max_length=12, # 문장의 토큰 기준 최대 길이. 문장이 12개 이상의 토큰으로 변환되면 잘림. 더 적은 수로 변환되면 패딩 토큰으로 채워진다.
    truncation=True, # 문장 잘림 허용 옵션
    return_tensors='tf'
)

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
batch_inputs['input_ids']

<tf.Tensor: shape=(3, 12), dtype=int32, numpy=
array([[ 9050,  9267,  7700,  9705, 23971, 12870,  8262,  7055,  7098,
         8084, 48213,     3],
       [19243, 29045,  8658, 11211, 11213,  9206,  7301, 14558,  8239,
        10765,  8052,  7621],
       [ 9686,  7445,   739, 26049,   389,     3,     3,     3,     3,
            3,     3,     3]], dtype=int32)>

In [ ]:
batch_inputs['attention_mask']

<tf.Tensor: shape=(3, 12), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
# text = '근육이 커지기 위해서는'
text = '피부의 조직이 괴사된다면'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

피부의 조직이 괴사된다면 그 원인은 아직 밝혀지지 않았다.
그러나 이 같은 사실은 이미 여러 차례 밝혀졌었다.
이번 연구에서는 뇌졸중 환자의 약 80%가 알츠하이머병이라는 사실을 밝혀냈다.
알츠는 신경전달물질인 아세틸콜린의 분비를 촉진해 뇌의 혈류량을 증가시킨다.
또한 혈관 내피세포의 기능을 활성화시켜 동맥경화를 예방한다.
뇌혈관이 좁아지면 혈액순환에 장애가 생겨 심혈관질환을 유발할 수 있다.
따라서 이번 연구는 심장마비나 당뇨병, 고혈압, 고지혈, 비만, 흡연, 음주, 흡연 등 다양한 위험요인을 가진 환자들에게 도움이 될 것으로 기대된다.
연구팀은 “심장


### BERT 입력값 만들기

- BERT (Bidirectional Encoder Representations from Transformers)는 다양한 자연어 처리(NLP) 작업에서 사용되는 인기 있는 모델. BertTokenizer는 특히 BERT 모델에 적합한 텍스트 토큰화 도구
- BERT 모델 입력값을 만들려면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 워드피스 어휘집합 구축 결과(`vocab.txt`)가 있어야 한다. 이미 만들어 놓은 워드피스 어휘집합을 포함한 BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언한다.

In [ ]:
# BertTokenizer는 BERT 모델이 이해할 수 있는 형태로 텍스트를 토큰화하는 데 사용
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(  # 사전 훈련된 토크나이저 로드
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case=False, # 토크나이저가 텍스트를 소문자로 변환하지 않도록 지정
)

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
print(tokenized_sentences)

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'], ['흠', '.', '.', '.', '포스터', '##보고', '초딩', '##영화', '##줄', '.', '.', '.', '.', '오버', '##연기', '##조차', '가볍', '##지', '않', '##구나'], ['별루', '였다', '.', '.']]


이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length", # max_length 매개변수로 지정된 최대 길이와 동일한 길이로 채워지도록 보장
    max_length=12, # 토큰화된 각 시퀀스가 ​​가져야 하는 고정 길이를 설정
    truncation=True, # 토크나이저가 max_length를 초과하는 시퀀스를 자를 수 있도록 허용
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [ ]:
# # BERT 모델 세 가지의 입력값
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

- [CLS] 토큰은 "분류"를 나타내며 모든 입력 시퀀스의 첫 번째 토큰으로 사용. BERT가 분류 작업(예: 감정 분석, 의도 감지)에 사용되는 경우 이 토큰의 표현은 분류 작업의 집계 시퀀스 표현으로 사용된다. 본질적으로 이는 전체 입력 시퀀스의 의미를 요약하는 역할
- [SEP] 토큰은 "구분 기호"를 나타내며 입력 내에서 개별 세그먼트를 구분하는 데 사용. 이는 질문 답변(모델이 질문과 맥락을 구별해야 하는 경우) 또는 문장 쌍 작업(예: 모델이 두 문장 간의 관계를 결정하는 자연어 추론)과 같은 여러 입력 시퀀스가 ​​포함된 작업에 특히 중요
- 예를 들어 문장 쌍 분류 작업에서 입력은 [CLS] 문장 1 [SEP] 문장 2 [SEP]와 같을 수 있으며 [SEP] 토큰은 첫 번째 문장의 끝을 표시하고 구분

In [ ]:
# 토큰 인덱스 시퀀스
# 문장 앞에 2, 끝에 3이 붙는 것은 각각 [CLS], [SEP] 라는 토큰에 대응하는 인덱스
batch_inputs['input_ids']

[[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1241, 16, 3],
 [2, 3274, 9509, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [ ]:
# BERT의 attention_mask는 GPT와 마찬가지로 일반 토큰이 자리한 곳(1)과 패딩 토큰이 자리한 곳(0)을 구분
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [ ]:
# token_type_ids는 세그먼트(segment)에 해당하는 것으로 모두 0
# BERT 모델은 기본적으로 문서(혹은 문장) 2개를 입력받는데, token_type_ids로 구분.
# 첫 번째 세그먼트(문서 혹은 문장)에 해당하는 token_type_ids는 0, 두 번째 세그먼트는 1
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

사전 훈련된 KoBERT 모델 사용

In [ ]:
from transformers import BertTokenizer

# KoBERT 모델의 사전 훈련된 토크나이저 로드
tokenizer_kobert = BertTokenizer.from_pretrained(
    "monologg/kobert",  # Hugging Face 모델 허브에서 KoBERT 모델을 지정
    do_lower_case=False,  # 토크나이저가 텍스트를 소문자로 변환하지 않도록 지정
)

# 사용 예시
text = "한국어 모델을 사용하여 텍스트 처리를 해봅시다."
input_ids = tokenizer_kobert.encode(text, add_special_tokens=True)
print(input_ids)


vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

[2, 0, 0, 0, 0, 0, 0, 54, 3]


In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_kobert.tokenize(sentence) for sentence in sentences]
print(tokenized_sentences)

[['아', '[UNK]', '.', '.', '진짜', '[UNK]', '[UNK]'], ['흠', '.', '.', '.', '[UNK]', '[UNK]', '.', '.', '.', '.', '[UNK]', '[UNK]', '[UNK]'], ['[UNK]', '였다', '.', '.']]


In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_kobert(
    sentences,
    padding="max_length", # max_length 매개변수로 지정된 최대 길이와 동일한 길이로 채워지도록 보장
    max_length=12, # 토큰화된 각 시퀀스가 ​​가져야 하는 고정 길이를 설정
    truncation=True, # 토크나이저가 max_length를 초과하는 시퀀스를 자를 수 있도록 허용
    return_tensors='tf'
)

batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(batch_inputs.input_ids)
print(batch_inputs.token_type_ids)
print(batch_inputs.attention_mask)

tf.Tensor(
[[   2 6797    0   54   54 7347    0    0    3    1    1    1]
 [   2 7989   54   54   54    0    0   54   54   54   54    3]
 [   2    0 6946   54   54    3    1    1    1    1    1    1]], shape=(3, 12), dtype=int32)
tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]], shape=(3, 12), dtype=int32)
tf.Tensor(
[[1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 0 0 0 0]], shape=(3, 12), dtype=int32)
